In [76]:
# http://metacyc.org/META/class-instances?object=Compounds
# Smart Table
# Add the following columns
# export with weblinks

In [1]:
import sys,os
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from metatlas import metatlas_objects as metob
import csv


curr_ld_lib_path = ''

os.environ['LD_LIBRARY_PATH'] = curr_ld_lib_path + ':/project/projectdirs/openmsi/jupyterhub_libs/boost_1_55_0/lib' + ':/project/projectdirs/openmsi/jupyterhub_libs/lib'
import sys
# sys.path.remove('/anaconda/lib/python2.7/site-packages')
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
sys.path.insert(0,'/project/projectdirs/openmsi/projects/meta-iq/pactolus/pactolus' )

from rdkit import Chem
# from rdkit.Chem.rdMolDescriptors import ExactMolWt
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem


In [2]:
metob.Compound()

{'creation_time': '2015-12-14T23:06:46',
 'description': u'No description',
 'formula': u'',
 'functional_sets': [],
 'head_id': u'c52a2a573175483e87d9398d894baf1e',
 'inchi': u'',
 'inchi_key': u'',
 'last_modified': '2015-12-14T23:06:46',
 'mono_isotopic_molecular_weight': 0.0,
 'name': u'Untitled',
 'neutralized_2d_inchi': u'',
 'neutralized_2d_inchi_key': u'',
 'neutralized_inchi': u'',
 'neutralized_inchi_key': u'',
 'number_components': 0,
 'permanent_charge': 0,
 'prev_uid': u'',
 'reference_xrefs': [],
 'synonyms': u'',
 'unique_id': u'c52a2a573175483e87d9398d894baf1e',
 'url': u'',
 'username': u'bpb'}

In [2]:
""" contribution from Hans de Winter """
def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None
def NeutraliseCharges(mol, reactions=None):
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
#     mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            rms_smiles = Chem.MolToSmiles(rms[0])
            mol = Chem.MolFromSmiles(rms_smiles)
    if replaced:
        return (mol, True) #Chem.MolToSmiles(mol,True)
    else:
        return (mol, False)

In [5]:
# datafile = '/global/homes/b/bpb/notebooks/bpb_data_analysis_metatlas/150624 ISTDs.txt'
# datafile = '/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/MetaCyc missing compounds (1).txt'
datafile = '/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/missing compounds not in pubchem.txt'
with open(datafile, 'rUb') as f:
    new_compounds = list(csv.DictReader(f, dialect='excel-tab'))
print new_compounds[0].keys()
new_compounds

 ['inchi', 'inchi_key', 'name', 'description']


[{'description': 'From Fiehn Lab Chemical Translation Service http://cts.fiehnlab.ucdavis.edu/',
  'inchi': 'InChI=1S/C10H13N5O5/c11-8-5-9(13-3-15(8)19)14(2-12-5)10-7(18)6(17)4(1-16)20-10/h2-4,6-7,10-11,16-19H,1H2/b11-8-\n',
  'inchi_key': 'QHFLZHVITNUFMV-FLIBITNWSA-N',
  'name': '6H-Purin-6-imine, 1,9-dihydro-1-hydroxy-9-pentofuranosyl-, (6Z)-\n(6Z)-6-Imino-9-pentofuranosyl-6,9-dihydro-1H-purin-1-ol\n'},
 {'description': 'From Fiehn Lab Chemical Translation Service http://cts.fiehnlab.ucdavis.edu/',
  'inchi': 'InChI=1S/C10H14O2/c1-9(2)6-4-7(11)10(9,3)8(12)5-6/h6H,4-5H2,1-3H3/t6-,10+\n',
  'inchi_key': 'PCEVUTCFZAQRKV-XGAOUMNUSA-N',
  'name': '1,7,7-Trimethylbicyclo[2.2.1]heptane-2,6-dione\nBicyclo[2.2.1]heptane-2,6-dione, 1,7,7-trimethyl-\n'},
 {'description': 'From Fiehn Lab Chemical Translation Service http://cts.fiehnlab.ucdavis.edu/',
  'inchi': 'InChI=1S/C10H14O3/c1-6-5-7-3-2-4-8(11)9(7)10(12)13-6/h6-7,12H,2-5H2,1H3\n',
  'inchi_key': 'GQKAFRQIXWEJOJ-UHFFFAOYSA-N',
  'name': '8H

In [14]:
for c in new_compounds:
    myInChI = str(c['inchi'])#'InChI=1S/C7H6O2/c8-7(9)6-4-2-1-3-5-6/h1-5H,(H,8,9)/i1D,2D,3D,4D,5D'
    if myInChI:
        myMol = Chem.MolFromInchi(myInChI.strip())
#         print c
#         print "mol = ",myMol
        print c['name']," ",Descriptors.ExactMolWt(myMol), " ",rdMolDescriptors.CalcMolFormula(myMol)
    else:
        print "not found"
            


6H-Purin-6-imine, 1,9-dihydro-1-hydroxy-9-pentofuranosyl-, (6Z)-
(6Z)-6-Imino-9-pentofuranosyl-6,9-dihydro-1H-purin-1-ol
  283.091668516   C10H13N5O5
1,7,7-Trimethylbicyclo[2.2.1]heptane-2,6-dione
Bicyclo[2.2.1]heptane-2,6-dione, 1,7,7-trimethyl-
  166.099379688   C10H14O2
8H-2-Benzopyran-8-one, 3,4,4a,5,6,7-hexahydro-1-hydroxy-3-methyl-
1-Hydroxy-3-methyl-3,4,4a,5,6,7-hexahydro-8H-isochromen-8-one
  182.094294308   C10H14O3
1,7,7-Trimethyltricyclo[2.2.1.0~2,6~]heptane
Tricyclo[2.2.1.0~2,6~]heptane, 1,7,7-trimethyl-
  136.125200512   C10H16
3-Aminopropanoyl 1-methyl-L-histidinate
  240.122240372   C10H16N4O3
1,3,3-Trimethyl-2-oxabicyclo[2.2.2]octane
2-Oxabicyclo[2.2.2]octane, 1,3,3-trimethyl-
  154.135765196   C10H18O
Methyl (2E)-2-[(1,4-dioxido-2-quinoxalinyl)methylene]hydrazinecarboxylate
Hydrazinecarboxylic acid, 2-[(1,4-dioxido-2-quinoxalinyl)methylene]-, methyl ester, (2E)-
  262.0702048   C11H10N4O4
2,3,5,8-Tetrahydroxy-6-methyl-1,4-naphthoquinone
1,4-Naphthalenedione, 2,3,5,8-te

In [21]:
all_compounds = []
for myCompound in new_compounds:
    c = metob.Compound()
    c.description = myCompound['description'].strip()
    c.name = myCompound['name'].strip()
    c.inchi = myCompound['inchi'].strip()
    c.inchi_key = myCompound['inchi_key'].strip()
    myMol = Chem.MolFromInchi(myCompound['inchi'].strip())
    if len(c.inchi) > 0:
        myCharge = Chem.GetFormalCharge(myMol)
        if  myCharge != 0:
            (myMol, neutralised) = NeutraliseCharges(myMol)
        c.formula = rdMolDescriptors.CalcMolFormula(myMol)
        c.mono_isotopic_molecular_weight = Descriptors.ExactMolWt(myMol)
        all_compounds.append(c)                
#     except:
#         print "Can not parse ", myCompound['name']


In [22]:
all_compounds

[{'creation_time': '2015-12-14T23:36:22',
  'description': u'From Fiehn Lab Chemical Translation Service http://cts.fiehnlab.ucdavis.edu/',
  'formula': u'C10H13N5O5',
  'functional_sets': [],
  'head_id': u'93cbf4e42f254856bc4d79ee676d742b',
  'inchi': u'InChI=1S/C10H13N5O5/c11-8-5-9(13-3-15(8)19)14(2-12-5)10-7(18)6(17)4(1-16)20-10/h2-4,6-7,10-11,16-19H,1H2/b11-8-',
  'inchi_key': u'QHFLZHVITNUFMV-FLIBITNWSA-N',
  'last_modified': '2015-12-14T23:36:22',
  'mono_isotopic_molecular_weight': 283.091668516,
  'name': u'6H-Purin-6-imine, 1,9-dihydro-1-hydroxy-9-pentofuranosyl-, (6Z)-\n(6Z)-6-Imino-9-pentofuranosyl-6,9-dihydro-1H-purin-1-ol',
  'neutralized_2d_inchi': u'',
  'neutralized_2d_inchi_key': u'',
  'neutralized_inchi': u'',
  'neutralized_inchi_key': u'',
  'number_components': 0,
  'permanent_charge': 0,
  'prev_uid': u'',
  'reference_xrefs': [],
  'synonyms': u'',
  'unique_id': u'93cbf4e42f254856bc4d79ee676d742b',
  'url': u'',
  'username': u'bpb'}, {'creation_time': '2015-1

In [23]:
#add them to the database
metob.store(all_compounds)

In [11]:
allCompounds = metob.retrieve('Compound', name='ABMBA')
for c in allCompounds:
    print c.name,c.formula,c.MonoIsotopic_molecular_weight,c.description, c.InChI

ABMBA C8H8BrNO2 228.974 https://pubchem.ncbi.nlm.nih.gov/compound/2774400 InChI=1S/C8H8BrNO2/c1-4-2-5(8(11)12)7(10)6(9)3-4/h2-3H,10H2,1H3,(H,11,12)


In [12]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [13]:
filename = os.path.basename(NOTEBOOK_FULL_PATH)
%system cp $filename /project/projectdirs/openmsi/www/
temp = '%s/%s'%('/project/projectdirs/openmsi/www',filename)
%system chmod 775 $temp
print 'http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/%s?flush_cache=true'%filename

http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/MetAtlas_003_Add_Compounds_To_Database.ipynb?flush_cache=true
